# Preliminary Training 

## 1. Load data set and initialize the Siamese Net

In [14]:
import torch 
import torchvision 
from torchvision import transforms

import datasets
import siamese_net

from imp import reload

In [37]:
train_data=datasets.Reddit_Img_Pair('.',transform=transforms.Compose([                      
                       transforms.Resize((224, 224)),
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]),re_select=True)
test_data=datasets.Reddit_Img_Pair('.',train=False, transform=transforms.Compose([                      
                       transforms.Resize((224, 224)),
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]),re_select=True)

In [40]:
train_loader = torch.utils.data.DataLoader(
    train_data, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    test_data, batch_size=64, shuffle=False)

In [20]:
net=siamese_net.SiameseNet(model_type=1)

In [21]:
net

SiameseNet(
  (branch): SiameseBranch(
    (locnet): LocNet(
      (alex_conv): Sequential(
        (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
        (1): ReLU(inplace)
        (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
        (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
        (4): ReLU(inplace)
        (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
        (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (7): ReLU(inplace)
        (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (9): ReLU(inplace)
        (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (11): ReLU(inplace)
        (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
      (extra_conv): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1))
      (fc_loc): Sequent

In [44]:
next(iter(train_loader))

[[tensor([[[[ 2.3760,  2.3887,  2.3760,  ...,  2.3760,  2.3760,  2.4015],
            [ 2.4015,  2.4142,  2.4015,  ...,  2.4269,  2.4015,  2.3760],
            [ 2.3760,  2.3887,  2.4015,  ...,  2.4778,  2.4269,  2.4651],
            ...,
            [ 2.3887,  2.3760,  2.4015,  ...,  2.4269,  2.4269,  2.4142],
            [ 2.3760,  2.3760,  2.4015,  ...,  2.4142,  2.4142,  2.3887],
            [ 2.3887,  2.4015,  2.4142,  ...,  2.4015,  2.3760,  2.3760]],
  
           [[ 0.8118,  0.8157,  0.8118,  ...,  0.8196,  0.8235,  0.8314],
            [ 0.8157,  0.8196,  0.8196,  ...,  0.8353,  0.8314,  0.8275],
            [ 0.8078,  0.8157,  0.8196,  ...,  0.8510,  0.8392,  0.8549],
            ...,
            [ 0.8275,  0.8235,  0.8314,  ...,  0.8314,  0.8314,  0.8275],
            [ 0.8235,  0.8235,  0.8314,  ...,  0.8275,  0.8275,  0.8196],
            [ 0.8275,  0.8314,  0.8353,  ...,  0.8235,  0.8157,  0.8157]],
  
           [[ 0.7490,  0.7529,  0.7490,  ...,  0.7922,  0.7843,  0.784

In [89]:
data,label=next(iter(train_loader))

In [90]:
output1,output2=net(data)

In [133]:
import losses
reload(losses)

<module 'losses' from '/Users/shuo/Desktop/Image-Virality/losses.py'>

In [86]:
criterion = losses.PairCrossEntropy()
# criterion = losses.ContrastiveLoss(1)


In [127]:
loss=criterion(output1,output2,1)

In [128]:
loss.backward()

RuntimeError: Trying to backward through the graph a second time, but the buffers have already been freed. Specify retain_graph=True when calling backward the first time.

Use the check point.

See the blog for details: https://byjiang.com/2017/06/05/How_To_Save_And_Restore_Model/

更好地使用预训练在自己地神经网络上，如果需要，冻结部分层。https://blog.csdn.net/e01528/article/details/85273172

断点训练。每次可以继续之前地训练。https://www.pytorchtutorial.com/pytorch-note5-save-and-restore-models/

In [162]:
def test(model, test_loader, device):
    with torch.no_grad():
        # it is encouraged to use the torch.no_grad to deactivate the auto-grad engine to speed up the computation
        model.eval()
        criterion = losses.PairCrossEntropy(size_average=False)
        test_loss = 0
        correct = 0
        for data, target in test_loader:
            data, target = [data_i.to(device)
                            for data_i in data], target.to(device)
            output1, output2 = model(data)

            # sum up batch loss
            test_loss += criterion(output1, output2,
                                   target)

            # get the prediction results
            pred_prob = torch.squeeze(criterion.get_log_probability(
                output1, output2))
            pred = np.zeros(data[0].size()[0])
#             print(pred_prob.numpy().shape)
            pred[pred_prob.numpy() > 0.5] = 1
#             print(target.numpy().shape)
            result_array = np.zeros(data[0].size()[0])
            result_array[pred == target.numpy()] = 1
            correct += result_array.sum()
            print("prediction:{},correction counts:{}".format(pred,correct))
        # get the average ranking loss and accuracy
        test_loss /= len(test_loader.dataset)
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'
              .format(test_loss, correct, len(test_loader.dataset),
                      100. * correct / len(test_loader.dataset)))

In [109]:
device = torch.device("cuda:0"
                          if torch.cuda.is_available() else "cpu")

In [163]:
test(net,test_loader,device)

prediction:[0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0.
 1. 0. 1. 1. 0. 1. 1. 0. 1. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 1. 1. 1. 0. 1.
 0. 1. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 1. 1.],correction counts:36.0
prediction:[1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 0. 1. 0. 1. 1. 1. 1. 0. 0. 1. 0. 0. 0. 1.
 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1.
 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 0. 1. 0. 1.],correction counts:71.0
prediction:[0. 0. 1. 1. 0. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1.
 0. 1. 1. 1. 0. 0. 1. 1. 0. 1. 0. 1. 1. 1. 0. 1. 0. 1. 0. 1. 0. 0. 1. 0.
 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1.],correction counts:94.0


KeyboardInterrupt: 